In [1]:
import pandas as pd #handle files, database etc.

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv") #read the csv file using pandas library

dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)

dataset = pd.get_dummies(dataset, drop_first=True) #"drop_first=True" removes 1st category of each categorical column

dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [4]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
#print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
#print(dataset[boolean_columns])

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset = dataset.drop("User ID", axis=1) #removing "User ID" feature using drop method. "axis=1" represent removing the "COLUMN"

dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts() #value_counts method calculates the purchased and not purchased value of "Purchased" column

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns #return column names present in the dataset variable

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [16]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']] #input features

dependent = dataset[['Purchased']] #output feature

In [19]:
#split training and testing dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {'n_neighbors':[5,10,15,20],
             'weights':['uniform','distance'],
             'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
             'p' : [1,2],
             'metric':['minkowski']}

grid = GridSearchCV(KNeighborsClassifier(), parameters, refit=True, n_jobs=-1, scoring='f1', verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'],
                         'n_neighbors': [5, 10, 15, 20], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1', verbose=3)

In [25]:
classifier = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(X_test) #test the model with test data

from sklearn.metrics import confusion_matrix #Evaluation metrics

cm = confusion_matrix(y_test, grid_pred)
print("Confusion Matrix : ", cm) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

Confusion Matrix :  [[73  6]
 [15 26]]
Best Score: 0.6887487046060901
Best Parameters: {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 15, 'p': 2, 'weights': 'uniform'}


In [27]:
table = pd.DataFrame.from_dict(classifier).dropna() #convert dictonary format to DataFrame(table) and dropna() method is used to avoid to display nan values rows

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026845,0.018948,0.015809,0.005729,auto,minkowski,5,1,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.682927,0.594595,0.628571,0.790698,0.682927,0.675943,0.066525,9
1,0.051798,0.035039,0.027084,0.009620,auto,minkowski,5,1,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.684211,0.631579,0.585366,0.750000,0.666667,0.663564,0.054855,25
2,0.054800,0.041581,0.016091,0.004933,auto,minkowski,5,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.682927,0.594595,0.628571,0.790698,0.650000,0.669358,0.067135,16
3,0.010483,0.002818,0.024315,0.005423,auto,minkowski,5,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.684211,0.631579,0.585366,0.750000,0.666667,0.663564,0.054855,25
4,0.010120,0.009683,0.019711,0.004473,auto,minkowski,10,1,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.648649,0.684211,0.482759,0.750000,0.529412,0.619006,0.098874,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.006491,0.005312,0.015346,0.003912,brute,minkowski,15,2,distance,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.702703,0.628571,0.585366,0.750000,0.651163,0.663561,0.057439,33
60,0.001061,0.002122,0.021942,0.006324,brute,minkowski,20,1,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.702703,0.687500,0.482759,0.736842,0.580645,0.638090,0.093590,49
61,0.004010,0.001619,0.011924,0.003537,brute,minkowski,20,1,distance,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.666667,0.628571,0.550000,0.750000,0.651163,0.649280,0.064398,41
62,0.004511,0.001455,0.018999,0.002126,brute,minkowski,20,2,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.702703,0.687500,0.482759,0.736842,0.580645,0.638090,0.093590,49


In [29]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_pred) #generation classification report

print(clf_report)

              precision    recall  f1-score   support

           0       0.83      0.92      0.87        79
           1       0.81      0.63      0.71        41

    accuracy                           0.82       120
   macro avg       0.82      0.78      0.79       120
weighted avg       0.82      0.82      0.82       120



In [31]:
#save the model using pickle library

import pickle
fileName = "finalized_model_knn_classification_grid.sav"
pickle.dump(grid, open(fileName, "wb")) #saved model available in regressor variable. Now storing that variable into fileName

In [33]:
loaded_model = pickle.load(open("finalized_model_knn_classification_grid.sav", "rb"))
result = loaded_model.predict([[30, 70000, 0]])

result

C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([0], dtype=int64)